In [ ]:
import os, glob, pickle

import numpy as np
import pandas as pd

from sklearn.metrics import auc

import matplotlib.pyplot as plt
import seaborn as sns

# Load Data and Generate AUCs
Loads in results generated from ZeroCrossingResults.ipynb, ZE Results.ipynb, and StaticWindowResults.ipynb and creates a master DataFrame containing all the scores for each of the three techniques explored.

In [ ]:
# Master DataFrame of Results
columns = ['method', 'window_size', 'step_size', 'SNR', 'trace_idx', 'AUC']
all_auc_df = pd.DataFrame(columns=columns)
all_auc_df.head()

In [ ]:
methods = ['sw', 'zcr', 'ae']
wss = [100, 500, 1000, 1500, 2000]
sss = [0.1, 0.5, 1.0]

# String template for locating result files
data_path_template = '{} ws/{} ws {} ss/{}_results_df.pkl'
data_path_template = os.path.join(os.getcwd(), data_path_template)
data_path_template           

In [ ]:
def auc_calc(group):
    group = group.sort_values(by='FPR')
    
    fprs = group.FPR.values
    tprs = group.TPR.values
    val = auc(fprs, tprs)
    
    return val

In [ ]:
for i, ws in enumerate(wss):
    for j, ss in enumerate(sss):
        for method in methods:
            #### Load in Results ####
            filename = data_path_template.format(ws, ws, ss, method)
            with open(filename, 'rb') as f:
                results = pickle.load(f)
                f.close()
            
            #### Calc AUC ####
            auc_df = results.groupby(['SNR', 'trace_idx']).apply(auc_calc).rename('AUC').to_frame()
            
            entries = [method, ws, ss]
            for l, col in enumerate(columns[:3]):
                auc_df[col] = entries[l]
            
            auc_df = auc_df.reset_index()
            all_auc_df = pd.concat((all_auc_df, auc_df))
all_auc_df      

# Plots
Visualise the effects that chaning window size and step size has to each anomaly detection technique.

In [ ]:
# Plot window size effect on AUC score
plot_snrs = [0.1, 0.25, 0.5, 1.0]
letters = ['(a)', '(b)', '(c)', '(d)']
fig, axs = plt.subplots(1, len(plot_snrs), figsize=(6, 2.5), sharey=True, dpi=600)

for i in range(len(plot_snrs)):
    plot_data = all_auc_df.loc[((all_auc_df.SNR == plot_snrs[i]) & (all_auc_df.step_size == 0.1))]
    sns.lineplot(data=plot_data, x='window_size', y='AUC', hue='method', ax=axs[i])
    axs[i].set_xlabel("Window Size", weight='bold')
    axs[i].annotate(letters[i], xy=(100, 0.5), xytext=(100, 1.08), weight='bold')
    axs[i].tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        axs[i].spines[axis].set_linewidth(1.5)  
    # axs[i].set(ylim=(0.5, 1.0))
    axs[i].legend().remove()
axs[0].set_ylabel('AUC', weight='bold')

fig.tight_layout()
fig.savefig("Window Size.png")

In [ ]:
# Plot step size effect on AUC score
plot_snrs = [0.1, 0.25, 0.5, 1.0]
letters = ['(a)', '(b)', '(c)', '(d)']
fig, axs = plt.subplots(1, len(plot_snrs), figsize=(6, 2.5), sharey=True, dpi=600)

for i in range(len(plot_snrs)):
    plot_data = all_auc_df.loc[((all_auc_df.SNR == plot_snrs[i]) & (all_auc_df.window_size == 500))]
    sns.lineplot(data=plot_data, x='step_size', y='AUC', hue='method', ax=axs[i])
    axs[i].set(ylim=(0.5, 1.0))
    axs[i].set_xlabel("Step Size", weight='bold')
    axs[i].legend().remove()
    axs[i].annotate(letters[i], xy=(0.5, 0.6), xytext=(0.1, 1.05), weight='bold')
    axs[i].tick_params(width=1.5)
    for axis in ['top', 'right', 'bottom', 'left']:
        axs[i].spines[axis].set_linewidth(1.5)
axs[0].set_ylabel('AUC', weight='bold')
fig.tight_layout()

fig.savefig("Step Size.png")

## Visulise both parameters in 3D

In [ ]:

plot_df = all_auc_df.groupby(['method', 'window_size', 'step_size', 'SNR']).agg('mean')
plot_df.head()

In [ ]:
SNRs = all_auc_df.SNR.unique()
        
fig, axs = plt.subplots(2, 2, figsize=(6, 4), dpi=600, subplot_kw={"projection": '3d'})

for method in methods:
    for i in range(axs.shape[0]):
        for j in range(axs.shape[1]):
            SNR = SNRs[i * 2 + j]

            z = np.zeros(x.shape)
            for k in range(x.shape[0]):
                for l in range(x.shape[1]):
                    z[k, l] = plot_df.loc[method, x[k, l], y[k, l], SNR]

            axs[i, j].plot_surface(x, y, z, alpha=0.5)
            axs[i, j].set_title('{} snr'.format(SNR))
            
            axs[i, j].set_xlim3d((0, 2000))
            axs[i, j].set_ylim3d((0, 1.0))
            axs[i, j].set_zlim3d((0.5, 1.0))
            
            if i == 0:
                axs[i, j].set_xticklabels([])
            if j == 0:
                axs[i, j].set_yticklabels([])
                axs[i, j].set_zticklabels([])
                     